## Análise de séries temporais com dados de aquecimento global
Dados disponíveis em: https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data

### Geração dos dados para análise

Importando bibliotecas:

In [1]:
from time import time, strftime, gmtime
global_start = time()

import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
from scipy import stats
from re import sub
from json import dump, load

import plotly.express as px
import plotly.graph_objects as go
from plotly import subplots
import matplotlib.pyplot as plt

#import sklearn.neighbors._base
#import sys
#sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
#from missingpy import MissForest
#from sklearn.preprocessing import LabelEncoder

Importando o arquivos com os dados por cidade:

In [2]:
df = pd.read_csv('../data/GlobalLandTemperaturesByCity.csv')

df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


Ajustando o tipo de dado da variável com as datas e separando a latitude para ser usada como medida de distância à linha do equador:

In [3]:
df.dt = pd.to_datetime(df.dt)

df['Latitude'] = df.Latitude.map(lambda x: float(x[:len(x)-1]))

df.head(3)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05,10.33E


Adicionando as temperaturas máxima e mínima com o erro.
+ Tmax = Temp + Erro
+ Tmin = Temp - Erro

In [4]:
df['max'] = df['AverageTemperature'] + df['AverageTemperatureUncertainty']
df['min'] = df['AverageTemperature'] - df['AverageTemperatureUncertainty']
df.head(3)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,max,min
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05,10.33E,7.805,4.331
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05,10.33E,NaN,NaN
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05,10.33E,NaN,NaN


Agrupando os dados por ano para diminuir o volume de dados e adicionando coluna de amplitude térmica (Tmáxima - Tmínima):

In [5]:
# Agrupando por ano, cidade, país e latitude, e calculando a mediana de cada registro gerado:

df_group = df.groupby(
    [df['dt'].map(lambda x: x.year), 'City', 'Country', 'Latitude']).agg(
    {'max': 'max', 'min': 'min'}).reset_index()

# Alterando os nomes para melhor manipulação:
df_group.columns = ['dt', 'City', 'Country', 'Latitude', 'max', 'min']

# Criando a coluna de amplitude térmica:
df_group['max_min'] = df_group['max'] - df_group['min']

# Organizando o dataframe por ordem de cidade e, em seguida, por ano:
df_group = df_group.sort_values(['City', 'Country', 'dt']).reset_index(drop=True)

Pela manipulação inicial foi visto que algumas cidades possuem mais de uma medida em algumas datas. Como as localizações são diferentes, deve se tratar de cidades maiores onde as medidas foram feitas em mais de um ponto.<br>
Os dados serão agrupados de novo, mas agora pela média:

In [6]:
# Contagem de registros por ano, cidade e país - existem cidades com mesmo nome em países diferentes:

pd.DataFrame(df_group[['dt', 'City', 'Country']].value_counts().reset_index()).head()

,dt,City,Country,0
0,1855,Rongcheng,China,3
1,1870,Rongcheng,China,3
2,1890,Springfield,United States,3
3,1831,Springfield,United States,3
4,2007,Springfield,United States,3


Agregação usando a média de cada conjunto de registros numéricos:

In [7]:
df_group = df_group.groupby(
    ['dt', 'City', 'Country']).agg(
    {'Latitude': 'mean', 'min': 'min', 'max': 'max', 'max_min': 'max'}).reset_index()

df_group = df_group.sort_values(['City', 'Country', 'dt']).reset_index(drop=True)

Os dados também possuem alguns valores NaN:

In [8]:
df_group[df_group['max_min'].isna()].head(10)

,dt,City,Country,Latitude,min,max,max_min
3,1746,A Coruña,Spain,42.59,NaN,NaN,NaN
4,1747,A Coruña,Spain,42.59,NaN,NaN,NaN
5,1748,A Coruña,Spain,42.59,NaN,NaN,NaN
6,1749,A Coruña,Spain,42.59,NaN,NaN,NaN
274,1746,Aachen,Germany,50.63,NaN,NaN,NaN
275,1747,Aachen,Germany,50.63,NaN,NaN,NaN
276,1748,Aachen,Germany,50.63,NaN,NaN,NaN
277,1749,Aachen,Germany,50.63,NaN,NaN,NaN
545,1746,Aalborg,Denmark,57.05,NaN,NaN,NaN
546,1747,Aalborg,Denmark,57.05,NaN,NaN,NaN


In [9]:
# Verificando número de cidades e número de cidades sem NaN para testar se há cidades com todos valores NaN:
print(df_group['City'].nunique(), df_group[df_group['max_min'].isna() == False]['City'].nunique())

3448 3448


Para não ser necessário a exclusão das cidades com valores NaN, será feita imputação nesses dados.<br>

Algumas opções:
+ imputação com missingpy.MissForest() criando dados a partir da aplicação de RandomForest;
+ interpolação.

**Opção 1)**

In [10]:
# A imputação com MissForest é extremamente exigente para o processador e nesse caso não é necessariamente uma
# escolha melhor do que a interpolação.
# Sendo assim, a opção 1 ficará apenas registrada abaixo:

#df_miss = df_group.copy()

#encoder = LabelEncoder()

#df_miss['City'] = encoder.fit_transform(df_miss['City'])
#df_miss['Country'] = encoder.fit_transform(df_miss['Country'])

#imputer = MissForest(max_iter=5)
#imp_data = pd.DataFrame(imputer.fit_transform(df_miss), columns=df_miss.columns.tolist())

#imp_data.head()

**Opção 2)**

In [11]:
df_int = df_group.set_index('dt', drop=True)

for n in df_int.columns.to_list():
    df_int[n] = df_int[n].interpolate()
    
for n in ['min', 'max', 'max_min']:
    df_int[n] = np.round(df_int[n], 2)
    
df_int.head(3)

,City,Country,Latitude,min,max,max_min
dt,,,,,,
1743,A Coruña,Spain,42.59,8.84,12.72,3.88
1744,A Coruña,Spain,42.59,6.76,19.84,13.08
1745,A Coruña,Spain,42.59,5.71,13.43,7.71


In [12]:
df_int.to_csv('../data/dados_ano_a_ano.csv')

Gravando os dados de países e cidades para a aplicação:

In [13]:
countries_cities = dict()
for n in df_int['Country'].sort_values().unique():
    country = n
    cities =  df_int[df_int['Country'] == n]['City'].sort_values().unique()
    countries_cities[country] = cities.tolist()

In [14]:
with open('../data/countries_cities.json', 'w', encoding='utf-8') as f:
    dump(countries_cities, f, ensure_ascii=False, indent=4)

In [15]:
f'Tempo de execução: {strftime("%H:%M:%S", gmtime(time()-global_start))}'

'Tempo de execução: 00:00:37'

### Fim